In [ ]:
# SKRYPT EKSPERYMENTALNY
# Uruchamia analizę algorytmu Tabu Search (TS) dla różnych
# konfiguracji parametrów. Dla każdej kombinacji wykonywane jest
# kilka powtórzeń, zapisywane są statystyki jakości oraz czas
# działania. Wyniki trafiają do pliku CSV.

%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
from src.utils.tsp_loader import load_tsp_matrix
from src.algorithms.tabu import solve_tsp
from src.utils.result_saver import save_experiment_results


# USTAWIENIA
TSP_FILES = ["Dane_TSP_76.xlsx"]

PARAM_GRID = {
    "max_iter": [100_000_000],
    "stop_no_improve": [100_000],
    "tabu_tenure": [25],
    "n_neighbors": [100],
    "neighborhood_type": ["two_opt"],
}

REPEATS = 3
results = []


# ROZGRZANIE (KOMPILACJA JIT JEŚLI WYSTĘPUJE)
print("Rozgrzewanie (kompilacja JIT jeśli dotyczy)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {"max_iter": 5, "tabu_tenure": 3, "n_neighbors": 5})
print("Kompilacja zakończona.\n")


# GŁÓWNA PĘTLA
start_total = time.perf_counter()

for tsp_file in TSP_FILES:
    print(f"\nInstancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for max_iter in PARAM_GRID["max_iter"]:
        for stop_no_improve in PARAM_GRID["stop_no_improve"]:
            for tabu_tenure in PARAM_GRID["tabu_tenure"]:
                for n_neighbors in PARAM_GRID["n_neighbors"]:
                    for neighborhood_type in PARAM_GRID["neighborhood_type"]:

                        costs = []
                        runtimes = []

                        for _ in range(REPEATS):
                            params = {
                                "max_iter": max_iter,
                                "stop_no_improve": stop_no_improve,
                                "tabu_tenure": tabu_tenure,
                                "n_neighbors": n_neighbors,
                                "neighborhood_type": neighborhood_type,
                            }

                            start_t = time.perf_counter()
                            route, cost, runtime, meta = solve_tsp(D, params)
                            total_time = time.perf_counter() - start_t

                            costs.append(cost)
                            runtimes.append(total_time)

                        results.append({
                            "instance": tsp_file,
                            "max_iter": max_iter,
                            "stop_no_improve": stop_no_improve,
                            "tabu_tenure": tabu_tenure,
                            "n_neighbors": n_neighbors,
                            "neighborhood_type": neighborhood_type,
                            "mean_cost": round(np.mean(costs), 3),
                            "min_cost": round(np.min(costs), 3),
                            "mean_runtime": np.mean(runtimes),
                        })


# ZAPIS WYNIKÓW
end_total = time.perf_counter()
elapsed = end_total - start_total

df = pd.DataFrame(results)
save_experiment_results(df, time_seconds=int(elapsed), subfolder="TS")


# RAPORT
print(f"\nŁączny czas eksperymentów: {elapsed/60:.2f} min ({elapsed:.2f} sek)")

print("\nPodsumowanie wyników:")
print(df.groupby("neighborhood_type")[["mean_cost", "min_cost", "mean_runtime"]]
      .mean()
      .round(3))

best = df.loc[df["mean_cost"].idxmin()]
print(f"\nNajlepsze parametry:\n{best.to_dict()}")

Rozgrzewanie (kompilacja JIT jeśli dotyczy)...
Kompilacja zakończona.


Instancja: Dane_TSP_76.xlsx
